# Capstone Project - The Battle of Neighborhoods

## Business Problem section

### Background

I live in Sydney, a place being in the world’s top-10 best cities for quality of living. Among the ranking elements is the dining culture. Indeed, Sydney is famous for its high-quality restaurants and is the place where you can find all of the world's famous dishes.

### Business Problem
One of my friends wanted to join the competition by investing in a restaurant business in inner Sydney (i.e. the City of Sydney). He asked me which suburb and which type of restaurants are the best to invest in. To answer the question, we can cluster the suburbs in inner Sydney with similar restaurant types. The target suburb should be the one in the cluster with the least number of restaurants.

The inner Sydney consists of 29 suburbs but our focus is on 5 most crowded suburbs - Sydney, The Rocks, Haymarket, Ultimo and Pyrmont.

## Data section



In [1]:
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [42]:
# read suburb list from 
df = pd.read_csv('https://opendata.arcgis.com/datasets/2a2b04faf74446309f7b22fd1d6651a2_0.csv')
df.head()

,FID,NAME,F2005_06,F2006_07,F2007_08,F2008_09,F2009_10,F2010_11,F2011_12,F2012_13,F2013_14,F2014_15,Shape_Leng,Shape_Area
0,1,Alexandria,179751048,179751048,1.803951e+08,1.644154e+08,1.542570e+08,1.623722e+08,1.631690e+08,155096161,1.455096e+08,147534614,10168.649178,3.523771e+06
1,2,Forest Lodge + Annandale,16720193,16720193,1.633630e+07,1.553792e+07,1.560317e+07,1.576891e+07,1.578564e+07,18743393,2.018737e+07,20761284,8654.226944,5.457704e+05
2,3,Millers Point + Barangaroo,39666586,39666586,4.135160e+07,4.160144e+07,4.184332e+07,4.059574e+07,3.791561e+07,34786136,3.014288e+07,37728668,3944.508809,4.634789e+05
3,4,Beaconsfield,8454492,8454492,1.012794e+07,1.192396e+07,1.233912e+07,1.284815e+07,1.251785e+07,9622120,5.212607e+06,5090894,1916.726468,1.674720e+05
4,5,Camperdown,116493273,116493273,1.195033e+08,1.225078e+08,1.260258e+08,1.267071e+08,1.245014e+08,129747022,1.334789e+08,139736392,7055.860737,1.072898e+06


In [43]:
df.drop(df.columns.difference(['NAME']), 1, inplace=True)
df.head()

,NAME
0,Alexandria
1,Forest Lodge + Annandale
2,Millers Point + Barangaroo
3,Beaconsfield
4,Camperdown


In [44]:
keep_list=['Sydney', 'The Rocks', 'Haymarket', 'Pyrmont', 'Ultimo']
df2=df[df['NAME'].isin(keep_list)]

In [45]:
df2

,NAME
14,Haymarket
19,Pyrmont
24,Sydney
25,The Rocks
26,Ultimo


In [50]:
geolocator = Nominatim(user_agent="Sydney_explorer")
df2['City_coord'] = df2['NAME'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [52]:
df2[['Latitude', 'Longitude']] = df2['City_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [55]:
df2 = df2.drop(columns=['City_coord'])

In [56]:
df2

,NAME,Latitude,Longitude
14,Haymarket,38.812140,-77.636804
19,Pyrmont,-33.869214,151.192363
24,Sydney,-33.854816,151.216454
25,The Rocks,-33.859992,151.208322
26,Ultimo,46.511994,10.899493


As you can see the coordinates of Haymarket and Ultimo are completely wrong, which is due similar suburb names in other countries so, I had to replace these coordinates with values acquired from google search.

In [66]:
df2['Latitude'][14] = -33.8809
df2['Longitude'][14] = 151.2029
df2['Latitude'][26] = -33.8822
df2['Longitude'][26] = 151.1970

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/l

In [67]:
df2

,NAME,Latitude,Longitude
14,Haymarket,-33.880900,151.202900
19,Pyrmont,-33.869214,151.192363
24,Sydney,-33.854816,151.216454
25,The Rocks,-33.859992,151.208322
26,Ultimo,-33.882200,151.197000


In [ ]:
address = 'Sydney, Australia'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

In [71]:
map_sydney = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, suburb in zip(df2['Latitude'], df2['Longitude'], df2['NAME']):
    label = '{}'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    
map_sydney

In [72]:
CLIENT_ID = 'TBSAWZA34Y1WK5SFX0LKDMURD2U1C3C5AWE1SWSVTE5J2AXJ' # your Foursquare ID
CLIENT_SECRET = '0B5O15IRU01GNF1N42ONWDZTTUPQFADEJNSF1RBKIHFCJEE5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TBSAWZA34Y1WK5SFX0LKDMURD2U1C3C5AWE1SWSVTE5J2AXJ
CLIENT_SECRET:0B5O15IRU01GNF1N42ONWDZTTUPQFADEJNSF1RBKIHFCJEE5


In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
sydney_venues = getNearbyVenues(names=df2['NAME'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

print("Shape of venue dataframe is ", sydney_venues.shape)
sydney_venues.head()

Haymarket
Pyrmont
Sydney
The Rocks
Ultimo
Shape of venue dataframe is  (278, 7)


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Haymarket,-33.8809,151.2029,Do Dee Paidang,-33.880670,151.203018,Thai Restaurant
1,Haymarket,-33.8809,151.2029,Satang Thai | สตางค์ (Satang Thai),-33.882317,151.203684,Thai Restaurant
2,Haymarket,-33.8809,151.2029,Ho Jiak,-33.879303,151.203520,Malay Restaurant
3,Haymarket,-33.8809,151.2029,Wake Up! Sydney,-33.882448,151.204836,Hostel
4,Haymarket,-33.8809,151.2029,Kura Japanese Dining,-33.880216,151.204244,Japanese Restaurant


In [80]:
sydney_venues.groupby('Suburb').count()

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Haymarket,71,71,71,71,71,71
Pyrmont,38,38,38,38,38,38
Sydney,14,14,14,14,14,14
The Rocks,100,100,100,100,100,100
Ultimo,55,55,55,55,55,55


In [82]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(sydney_venues['Venue Category'].unique())))

There are 89 uniques categories.


In [83]:
# one hot encoding
venues_onehot = pd.get_dummies(sydney_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Suburb'] = sydney_venues['Suburb'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Suburb,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Court,Bistro,...,Supermarket,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Theater,Tour Provider,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Haymarket,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Haymarket,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Haymarket,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Haymarket,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Haymarket,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
sydney_grouped = venues_onehot.groupby('Suburb').mean().reset_index()
sydney_grouped

,Suburb,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Court,Bistro,...,Supermarket,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Theater,Tour Provider,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Haymarket,0.000000,0.00,0.014085,0.000000,0.014085,0.014085,0.014085,0.000000,0.00,...,0.000000,0.014085,0.014085,0.154930,0.014085,0.00,0.00,0.000000,0.014085,0.00
1,Pyrmont,0.000000,0.00,0.026316,0.026316,0.000000,0.026316,0.105263,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.026316,0.00,0.00,0.000000,0.026316,0.00
2,Sydney,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.214286,0.00,0.00,0.000000,0.000000,0.00
3,The Rocks,0.000000,0.01,0.010000,0.060000,0.000000,0.020000,0.010000,0.000000,0.01,...,0.000000,0.000000,0.000000,0.010000,0.010000,0.01,0.01,0.000000,0.000000,0.01
4,Ultimo,0.018182,0.00,0.000000,0.018182,0.000000,0.000000,0.036364,0.018182,0.00,...,0.036364,0.000000,0.000000,0.018182,0.000000,0.00,0.00,0.018182,0.000000,0.00


In [90]:
# What are the top 5 venues/facilities nearby profitable real estate investments?

num_top_venues = 5

for hood in sydney_grouped['Suburb']:
    print("----"+hood+"----")
    temp = sydney_grouped[sydney_grouped['Suburb'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

----Haymarket----
----Pyrmont----
----Sydney----
----The Rocks----
----Ultimo----


In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [92]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Suburb'] = sydney_grouped['Suburb']

for ind in np.arange(sydney_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(sydney_grouped.iloc[ind, :], num_top_venues)

In [94]:
venues_sorted.head()

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Haymarket,Thai Restaurant,Chinese Restaurant,Coffee Shop,Café,Hostel,Korean Restaurant,Ice Cream Shop,Japanese Restaurant,Malay Restaurant,Hotpot Restaurant
1,Pyrmont,Café,Bar,Fish Market,Seafood Restaurant,Pub,Restaurant,Japanese Restaurant,Breakfast Spot,Butcher,Rock Club
2,Sydney,Theater,Concert Hall,Australian Restaurant,Cocktail Bar,French Restaurant,Park,Opera House,Café,Plaza,German Restaurant
3,The Rocks,Café,Hotel,Australian Restaurant,Pub,Cocktail Bar,Italian Restaurant,Park,Museum,Brewery,Hotel Bar
4,Ultimo,Café,Coffee Shop,Dumpling Restaurant,Hotel,Ice Cream Shop,Supermarket,Bar,Art Gallery,Museum,Multiplex


In [ ]:
#Distribute in 3 Clusters

# set number of clusters
kclusters = 3

sydney_grouped_clustering = sydney_grouped.drop('Suburb', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sydney_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]